In [ ]:
# for Colab use

# ! git clone https://github.com/stevenabreu7/hybrid-interpretability
# ! cd hybrid-interpretability
# ! bash install.sh

In [ ]:
import kagglehub
import os

kagglehub.login()

In [ ]:
# import model weights

google_recurrentgemma_pytorch_2b_it_1_path = kagglehub.model_download('google/recurrentgemma/PyTorch/2b/1')

## Installation

In [ ]:
#@title Imports
import pathlib
import torch

import sentencepiece as spm
from recurrentgemma import torch as recurrentgemma

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
google_recurrentgemma_pytorch_2b_it_1_path="/root/.cache/kagglehub/models/google/recurrentgemma/PyTorch/2b/1"

In [ ]:
VARIANT = '2b'
weights_dir = pathlib.Path(f"{google_recurrentgemma_pytorch_2b_it_1_path}")
ckpt_path = weights_dir / f'{VARIANT}.pt'
vocab_path = weights_dir / 'tokenizer.model'
preset = recurrentgemma.Preset.RECURRENT_GEMMA_2B_V1 if '2b' in VARIANT else recurrentgemma.Preset.RECURRENT_GEMMA_9B_V1

### Load and prepare RG

In [ ]:
# Load parameters
params = torch.load(str(ckpt_path))
params = {k : v.to(device=device) for k, v in params.items()}

<ipython-input-5-3ea271273971>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(str(ckpt_path))


In [ ]:
model_config = recurrentgemma.GriffinConfig.from_torch_params(
    params,
    preset=preset,
)
model = recurrentgemma.Griffin(model_config, device=device, dtype=torch.bfloat16)
model.load_state_dict(params)

<All keys matched successfully>

In [ ]:
model.enable_sparsification(k = 3, metric = "entropy", prefill = False)

In [ ]:
vocab = spm.SentencePieceProcessor()
vocab.Load(str(vocab_path))

True

In [ ]:
sampler = recurrentgemma.Sampler(model=model, vocab=vocab)

### generate

In [ ]:
input_batch = ["I once had a girl, or should I say, she once had  "]

# 30 generation steps
out_data = sampler(input_strings=input_batch, total_generation_steps=30)

for input_string, out_string in zip(input_batch, out_data.text):
  print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")
  print(10*'#')

Prompt:
I once had a girl, or should I say, she once had  
Output:
me.

I was a young girl, a young girl, a young girl, a young girl, a young girl, a young girl, a
##########


## Testing attention sparsification

In [ ]:
input_batch = ["I once had a girl, or should I say, she once had "]

model.disable_attention_manipulation()
model.enable_sparsification(k = 3, metric = "entropy", prefill = False)

# 30 generation steps
out_data = sampler(input_strings=input_batch, total_generation_steps=30)

for input_string, out_string in zip(input_batch, out_data.text):
  print(f"Prompt:\n{input_string}\nOutput:\n{out_string}")
  print(10*'#')

Prompt:
I once had a girl, or should I say, she once had  
Output:
me.

I was a young man, a young man with a lot of potential. I was a young man with a lot of potential, and
##########


##NIAH

###Loading NIAH

In [ ]:
%cd /content/

/content


In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
!pwd

In [ ]:
!pip install python-dotenv tiktoken anthropic

In [ ]:
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
!python NIAH/Needle_test/prompt.py

📝 Loading configuration...
🔧 Creating Prompter instance...
🚀 Initializing Prompter...
📁 Haystack directory: LongAlign/Needle_test/PaulGrahamEssays
💾 Save directory: LongAlign/Needle_test/prompts
⚙️ Setting up context lengths...
✅ Context lengths set: [ 250  677 1105 1532 1959 2387 2814 3241 3669 4096]
⚙️ Setting up document depth percentages...
✅ Depth percentages set: [  0  11  22  33  44  56  67  78  89 100]
🔄 Initializing Huggingface tokenizer...
🤗 Initializing HuggingFace tokenizer...
🌐 Loading tokenizer from HuggingFace Hub: models/recurrent_gemma_default
❌ HuggingFace tokenizer initialization failed: models/recurrent_gemma_default is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
❌ Tokenizer initialization failed: models/recurrent_gemma_default is not a local

###Run predictions

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python NIAH/Needle_test/pred.py

🔹 Loading configuration...
🔹 Prompt directory (relative): LongAlign/Needle_test/prompts
🔹 Prompt directory (absolute): /content/LongAlign/Needle_test/prompts
🔹 Model provider: Huggingface
/content/LongAlign/Needle_test/adjusted/pred.py:183: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full 

###Eval and Viz

make sure to set api key before running in eval.py

In [ ]:
%cd /content/

/content


In [ ]:
import subprocess
import os
from google.colab import userdata
my_env = os.environ.copy()
hf_token = userdata.get('HF_TOKEN')
custom_args = dict(token=hf_token)
custom_args = dict(token=hf_token, cache_dir="./.cache")  # speeds up loading
my_env["CUDA_VISIBLE_DEVICES"] = "0"  # adjust for multi-gpu

In [ ]:
!python NIAH/Needle_test/eval.py

Found K values: [0, 1, 2, 10]

Evaluation Summary:

K0:
  Total predictions: 100
  Successful predictions: 0
  Success rate: 0.00%

K1:
  Total predictions: 100
  Successful predictions: 1
  Success rate: 1.00%

K2:
  Total predictions: 100
  Successful predictions: 50
  Success rate: 50.00%

K10:
  Total predictions: 100
  Successful predictions: 58
  Success rate: 58.00%


In [ ]:
!python LongAlign/Needle_test/vis.py

Found JSON files: ['LongAlign/Needle_test/results/binary_eval_all_k.json']

Processing file: LongAlign/Needle_test/results/binary_eval_all_k.json
Processing K0
Processing K1
Processing K2
Processing K10


In [ ]:
!zip -r /content/prompt.zip /content/LongAlign/Needle_test/prompts
from google.colab import files
files.download("/content/PRED/prompt.zip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -r pred_K10
!ls

adjusted	  config-prompt.yaml		    pred_K1	 pred.py     README.md
config-eval.yaml  eval.py			    pred_K1.zip  prompt.py   vis.py
config-pred.yaml  llama_flash_attn_monkey_patch.py  pred_K2	 prompts
config_pred.yaml  PaulGrahamEssays		    pred_K2.zip  prompt.zip


In [ ]:
!zip -r /content/pred_K1.zip /content/LongAlign/Needle_test/pred_K1
files.download("/content/pred_K1.zip")

In [ ]:
from google.colab import files

!zip -r /content/pred_K2.zip /content/LongAlign/Needle_test/pred_K2
files.download("/content/pred_K2.zip")

In [ ]:
!zip -r /content/results.zip /content/LongAlign/Needle_test/results
files.download("/content/results.zip")